In [3]:
# Image recognition app using tensorflow and keras to determine if a webcam video is one of 10 categories

# Import libraries
import tensorflow as tf
from tensorflow import keras
# For image manipulation
import PIL
import pathlib
# For linear algebra
import numpy as np
import matplotlib.pyplot as plt

# Class names for the 10 different categories identifying whether a driver is impaired or not
class_names = ['Safe_driver', 'Texting_right', 'Calling_right', 'Texting_left', 'Calling_left',
               'Infotainment', 'Drinking', 'Reaching', 'Scratching', 'Head_turned']

num_classes = 10

# Image paths
data_dir = pathlib.Path('imgs')
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)
print(data_dir)
test_dir = pathlib.Path('imgs/testing')
train_dir = pathlib.Path('imgs/train')

batch_size = 32
# Image size used throughout the process
img_height, img_width = 480, 640

# Number of epochs to train the model
epochs = 15

# Load the data
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    labels="inferred",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="validation",
    labels="inferred",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels="inferred",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

# For testing purposes
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

hello
0
imgs
Found 22424 files belonging to 10 classes.
Using 17940 files for training.


2023-01-28 13:14:31.891679: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 22424 files belonging to 10 classes.
Using 4484 files for validation.
Found 79726 files belonging to 1 classes.
(32, 480, 640, 3)
(32,)


In [ ]:
# Configure the dataset for performance
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Normalize the data
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1. / 255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

# Create the model
model = tf.keras.Sequential([
    # Normalization layer
    tf.keras.layers.experimental.preprocessing.Rescaling(1. / 255, input_shape=(img_height, img_width, 3)),
    # Convolutional layers
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 128 neuron hidden layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes)
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit_generator(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=2
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_ds, verbose=2)
print('Test accuracy:', test_acc)

# Save the model
model.save('model.h5')

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-01-28 13:15:29.183612: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 32 of 1000
2023-01-28 13:15:40.986210: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 47 of 1000
2023-01-28 13:15:49.738185: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 54 of 1000
2023-01-28 13:15:59.594971: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 66 of 1000
2023-01-28 13:16:10.255253: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 72 of 1000
2023-01-28 13:16:18.986795: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 80 of 1000
2023-01-28 13:16:29.442430: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer 